# Table of Contents

This notebook generates data used to contruct `binding peak` tables and `venn diagrams` for ChipDB. But before you run the notebook, make sure of the following: 

1. Concatnate files that contain multiple conditions for the same TF by using the following command on your terminal: `cat /path/prefix* >> new_file`.
2. Make sure to rename TF in the gff file to match the TF convention and not gene convension.

In [1]:
#basics:
import numpy as np     
import pandas as pd
import urllib                      
import gzip
import os
from os import path
from collections import defaultdict

# Binding Peak Tables

## Generate a data objects

### Prerequisit
Ensure the following paths reflect your local **input** and **output** directories

In [2]:
data_dir = abspath(join('..','plot_dev', 'data_generation'))

NameError: name 'abspath' is not defined

In [3]:
in_data = join(data_dir, 'input_data')
in_gff = join(in_data, 'gff_data')
out_data_venn = join(data_dir, 'output_data','venn_data')
out_data_table = join(data_dir, 'output_data','table_data')

NameError: name 'join' is not defined

In [17]:
database = defaultdict(list)

In [18]:
filelist = [file for file in os.listdir(int_dir)]

for i in range(0,len(filelist)):
    df = pd.read_csv(path.join(int_dir,filelist[i]),index_col=0, 
                                 delimiter='\t', header=None, 
                                 names = ['ref','condition', 'condition_name', 
                                          "binding_peak_start",'binding_peak_end', 
                                          'binding_peak_strength', 'direction', '.','ID'])

    df = df.set_index(pd.Series(range(1,len(df)+1)))
    database[i] = df

In [19]:
TF_list =[]
TF_gene = []
for i in range (0,len(database)): 
    TF_list.append(database[i].condition_name[1][:4])
    TF_gene.append(database[i].condition_name[2][:4])

In [31]:
TF_list

['YheO',
 'YbiH',
 'YbaQ',
 'YdcI',
 'YbaO',
 'YieP',
 'Yagi',
 'YiaJ',
 'YddM',
 'YafC',
 'YeiE']

## Load gene info + Biocyc TRN 

In [20]:
DF_EcoCyc_TF = pd.read_csv(path.join(data_dir,'ecocyc_trn_data.csv'),index_col=0)
DF_gene_info = pd.read_csv(path.join(data_dir,'gene_info.csv'),index_col=0)
precise2_TRN = pd.read_csv(path.join(data_dir,'TRN.csv'),index_col=0)

In [21]:
# Define start codon position (to account for +/- strands)
DF_gene_info['start_codon_pos'] = [row.start if row.strand == '+' else row.stop for idx,row in DF_gene_info.iterrows()]
DF_gene_info.head()

,start,stop,strand,gene_name,length,operon,cog,start_codon_pos
b0001,189,255,+,thrL,66,thrLABC,No COG Annotation,189
b0002,336,2799,+,thrA,2463,thrLABC,No COG Annotation,336
b0003,2800,3733,+,thrB,933,thrLABC,Amino acid transport and metabolism,2800
b0004,3733,5020,+,thrC,1287,thrLABC,Amino acid transport and metabolism,3733
b0005,5233,5530,+,yaaX,297,yaaX,Function unknown,5233
...,...,...,...,...,...,...,...,...
b4399,4636695,4638120,+,creC,1425,creABCD,Signal transduction mechanisms,4636695
b4400,4638177,4639530,+,creD,1353,creABCD,Defense mechanisms,4638177
b4401,4639589,4640306,-,arcA,717,arcA,Signal transduction mechanisms,4640306
b4402,4640401,4640542,+,yjjY,141,yjjY,Function unknown,4640401


## Generate Binding peaks 

This function assigns bindings peaks for each TF and fins the corresponsing gene targets using the `gene_info.csv` table. 

In [22]:
def annotate_peaks(peak_df,margin,gene_info):
    
    res_df = peak_df.copy()
    TF = peak_df.condition_name[1][:4]
    for i,row in res_df.iterrows():
        pos = row['binding_peak_start']
        # Identify genes within MARGIN nt of binding peak
        close_genes = gene_info[(gene_info.start_codon_pos > pos-margin) 
                                & (gene_info.start_codon_pos < pos+margin)]
        for strand,group in close_genes.groupby('strand'):
            #Remove genes that are completely transcribed before binding peak
            if strand == '+':
                group = group[group.stop > pos]
            else:
                group = group[group.start < pos]

            operon = group.operon.unique()
            # Ensure that we're only identifying one operon on either side of binding peak
#             if len(operon) > 1:
#                 print (operon)

            # Get all genes in operon
            bnums = gene_info[gene_info.operon.isin(operon)].index
            
            ## Add gene information to dataframe
            if strand == '+':
                res_df.loc[i,'TU_p'] = ','.join(operon)
                res_df.loc[i,'genes_p'] = ','.join(bnums)
            else:
                res_df.loc[i,'TU_m'] = ','.join(operon)
                res_df.loc[i,'genes_m'] = ','.join(bnums)
    res_df['index'] = [TF +'-' + str(i) for i in range(1,peak_df.shape[0]+1)]
    res_df['condition'] = [peak_df.condition_name[2][:4]+' + '+peak_df.condition_name[1][5:8]]*peak_df.shape[0]
    cols = ['index','condition','binding_peak_start','binding_peak_end',
            'binding_peak_strength','TU_p','genes_p','TU_m','genes_m']
    return res_df.reindex(columns = cols)

## Validate Binding Peaks 

this function validates the accuracy of every gene target from the already indetified gene list for every binding site, identified from the previous function  

In [24]:
def validate_peak_info(df,gene_info):
    locusTag = defaultdict(list)
    geneName = defaultdict(list)
    for i,row in df.iterrows():
        BP = row['binding_peak_start']
        idx_name = i
        genes = [row['genes_p'] , row['genes_m']]
        if ((genes[0] == '') & (genes[1] == '')):
            locusTag[idx_name].append('')
            geneName[idx_name].append('')
        for gene in genes:
            if gene == '':
                continue 
            gene_list = gene.split(',')
            for g in gene_list: 
                name = DF_gene_info.loc[g].gene_name 
                strand = DF_gene_info.loc[g].strand 
                start = DF_gene_info.loc[g].start
                stop = DF_gene_info.loc[g].stop
                if ((start > BP) & (stop > BP) & (strand == '+')) | ((start < BP) & (stop < BP) & (strand == '-')):
                        locusTag[idx_name].append(g)
                        geneName[idx_name].append(name)
                elif ((start < BP) & (stop > BP)) | ((start > BP) & (stop < BP)):
                    locusTag[idx_name].append(g)
                    geneName[idx_name].append(name)
                    
    for k, v in locusTag.items():
        if ((len(v) == 1) & (v[0] == '')): 
            locusTag[k] = ''
            continue
        genes = ','.join(locusTag[k])
        locusTag[k] = genes

    for k, v in geneName.items():
        if ((len(v) == 1) & (v[0] == '')): 
            geneName[k] = ''
            continue
        genes = ','.join(geneName[k])
        geneName[k] = genes

    df_complete = df.loc[:,['index','condition','binding_peak_start',
                            'binding_peak_end','binding_peak_strength']]
    df_complete['target_locus'] = locusTag.values()
    df_complete['target_genes'] = geneName.values()
    return df_complete

# Venn Diagrams

Make sure to update the list of `TF_names` as Ye adds more gff files into the dropbox

In [26]:
TRN_data = defaultdict(list)
TF_name = [
 'YheO',
 'YbiH',
 'YbaQ',
 'YbaO',
 'YieP',
 'YagI',
 'YiaJ',
 'YddM',
 'YdcI',
 'YafC',
 'YeiE']

for i in TF_name: 
    TRN_data[i] = [x for x in  list(precise2_TRN.gene_name[precise2_TRN.index == i])  if  str(x) != 'nan']

In [27]:
def Venn_data_gen(Peak_DF): 
    TF_name = Peak_DF['index'][1][:4]
    gene_list = [i for i in Peak_DF.target_genes if i != '']
    chip_data= ','.join(list(gene_list)).split(',')
    reg_data = TRN_data[TF_name]
    all_genes = [i  for i in chip_data if i not in reg_data] + reg_data
    
    
    TF = TF_name
    reg_genes=reg_data
    reg_only = []
    chip_genes=chip_data
    chip_only = []
    shared_genes=[]
    for i in all_genes: 
        if (i in reg_data) & (i not in chip_data):
            reg_only.append(i)
        elif (i in chip_data) & (i not in reg_data):
            chip_only.append(i)
        elif (i in chip_data) & (i in reg_data):
            shared_genes.append(i)
            
    values = [TF,
          len(reg_genes),
          len(reg_only),
          len(chip_genes),
          len(chip_only),
          len(shared_genes),
          len(all_genes)]
    
    index_name = ['TF',
              'reg_genes',
              'reg_only',
              'chip_genes',
              'chip_only',
              'shared_genes',
              'all_genes']
    
    genes = ['',
         reg_genes,
         reg_only,
         chip_genes,
         chip_only,
         shared_genes,
         all_genes]
    
    same1 = defaultdict(list)
    for i in range(0,len(index_name)):
        same1[index_name[i]].append(values[i])

    finall = pd.DataFrame.from_dict(same1, orient='index', columns = ['value'])
    finall['list'] = genes
    for i, row in finall.iterrows(): 
        if row.value == 0:
            finall.list[i] = ''
    return finall

## Generate the data

Run this block if you are ready to generate your data for ChIPdb.

In [29]:
for i in range (0, len(database)): 
    peak_annot_DF = annotate_peaks(database[i],500,DF_gene_info).fillna('')
    final_annot_DF = validate_peak_info(peak_annot_DF,DF_gene_info)
    venn_files = Venn_data_gen(final_annot_DF)
    final_annot_DF.to_json(path.join(outjson_dir, TF_list[i]+'.json'),orient='records')
    venn_files.to_json(path.join(outcsv_dir, TF_list[i]+'.json'),orient='records')